In [1]:
# Autor: Alejandro Gerena
# Objetivo: Aplicar limpieza de datos según problemas identificados y 
#           exportar a formato JSON estructurado
# ============================================================================

import pandas as pd
import numpy as np
import json
from datetime import datetime
import re
import os
import csv

In [2]:
# ============================================================================
# ETAPA 1: CARGA Y EXPLORACIÓN INICIAL
# ============================================================================

def cargar_dataset(filename: str) -> pd.DataFrame | None:
    try:
        base_dir = os.getcwd()
        file_path = os.path.join(base_dir, "..", "data", filename)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"El archivo {file_path} no existe.")

        ext = os.path.splitext(filename)[1].lower()

        if ext == ".csv":
            # Intentar abrir con UTF-8, si falla usar latin-1
            for encoding in ["utf-8", "latin-1"]:
                try:
                    with open(file_path, 'r', encoding=encoding) as f:
                        sample = f.read(2048)
                        sniffer = csv.Sniffer()
                        dialect = sniffer.sniff(sample)
                        detected_sep = dialect.delimiter

                    data = pd.read_csv(file_path, encoding=encoding, sep=detected_sep)
                    break  # Si carga correctamente, salir del bucle
                except UnicodeDecodeError:
                    continue
            else:
                raise UnicodeDecodeError("No se pudo decodificar el archivo con utf-8 ni latin-1")

        elif ext in [".xls", ".xlsx"]:
            data = pd.read_excel(file_path)
        else:
            raise ValueError("Formato de archivo no soportado. Use CSV o Excel (.xls, .xlsx).")

        print(f">>> El archivo '{filename}' se cargó correctamente. Filas: {data.shape[0]}, Columnas: {data.shape[1]}")
        return data

    except FileNotFoundError as e:
        print(e)
    except pd.errors.ParserError as e:
        print(f"Error de formato: {e}")
    except ValueError as e:
        print(e)
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

    return None


def explorar_dataset(df):
    """
    Muestra información básica del dataset
    """
    print("\n" + "="*80)
    print("EXPLORACIÓN INICIAL DEL DATASET")
    print("="*80)
    
    print("\n1. Primeras filas:")
    display(df.head(3))
    
    print("\n2. Información de columnas:")
    print(df.info())
    
    print("\n3. Valores nulos por columna:")
    print(df.isnull().sum())
    
    print("\n4. Valores únicos en columnas clave:")
    for col in ['flightNumber', 'plane', 'dep_airport_code', 'arr_airport_code']:
        if col in df.columns:
            print(f"  - {col}: {df[col].nunique()} valores únicos")
    
    return df

In [3]:
# Ejecutar carga y exploración
# Cargar información de vuelos
df_raw = cargar_dataset("infovuelos_sample.csv")
df_raw = explorar_dataset(df_raw)

>>> El archivo 'infovuelos_sample.csv' se cargó correctamente. Filas: 39102, Columnas: 25

EXPLORACIÓN INICIAL DEL DATASET

1. Primeras filas:


,flightNumber,plane,dep_date,dep_time,dep_airport_name,dep_airport_code,dep_terminal,dep_status,dep_weather_min,dep_weather_max,...,arr_airport_name,arr_airport_code,arr_terminal,arr_status,arr_weather_min,arr_weather_max,arr_weather_desc,arr_room,arr_belt,timestamp
0,HTY134,AWH,13/04/18,10:00,ALGECIRAS / HELIPUERTO,AEI,-,El vuelo ha despegado a las 10:03,9,16,...,CEUTA,JCU,-,El vuelo ha aterrizado a las 10:12,11,17,Lluvia,-,-,2018-04-13 10:33:45
1,HTY110,AWH,13/04/18,14:15,CEUTA,JCU,-,Llegada prevista a las 14:15,11,17,...,ALGECIRAS / HELIPUERTO,AEI,-,Salida prevista a las 14:35,9,16,Chubascos dispersos,-,-,2018-04-13 10:33:47
2,HTY112,AWH,13/04/18,15:05,ALGECIRAS / HELIPUERTO,AEI,-,Salida prevista a las 15:10,9,16,...,CEUTA,JCU,-,Llegada prevista a las 15:15,11,17,Lluvia,-,-,2018-04-13 10:33:48



2. Información de columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39102 entries, 0 to 39101
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   flightNumber      39102 non-null  object
 1   plane             39102 non-null  object
 2   dep_date          39102 non-null  object
 3   dep_time          39102 non-null  object
 4   dep_airport_name  39102 non-null  object
 5   dep_airport_code  39102 non-null  object
 6   dep_terminal      39102 non-null  object
 7   dep_status        39102 non-null  object
 8   dep_weather_min   39102 non-null  object
 9   dep_weather_max   39102 non-null  object
 10  dep_weather_desc  39102 non-null  object
 11  dep_counter       39102 non-null  object
 12  dep_door          39102 non-null  object
 13  arr_date          39102 non-null  object
 14  arr_time          39102 non-null  object
 15  arr_airport_name  39102 non-null  object
 16  arr_airport_code  39102 non-n

In [4]:
# ============================================================================
# ETAPA 2: LIMPIEZA - PROBLEMA 1 (Valores nulos representados como "-")
# ============================================================================

def detectar_problema_1(df):
    """
    Detecta columnas con valores "-" que deberían ser nulos
    """
    print("\n" + "="*80)
    print("PROBLEMA 1: Detección de valores '-' como nulos")
    print("="*80)
    
    columnas_con_guion = []
    for col in df.columns:
        count_guion = (df[col] == '-').sum()
        if count_guion > 0:
            columnas_con_guion.append((col, count_guion))
            print(f"  - {col}: {count_guion} valores '-'")
    
    return columnas_con_guion


def limpiar_problema_1(df):
    """
    Reemplaza valores "-" por NaN/None
    """
    df_clean = df.copy()
    df_clean.replace('-', np.nan, inplace=True)
    print("\n✓ Valores '-' reemplazados por NaN")
    return df_clean


def validar_problema_1(df_original, df_limpio):
    """
    Valida que los "-" se hayan eliminado correctamente
    """
    print("\n" + "-"*80)
    print("VALIDACIÓN PROBLEMA 1")
    print("-"*80)
    
    guiones_antes = (df_original == '-').sum().sum()
    guiones_despues = (df_limpio == '-').sum().sum()
    
    print(f"  Valores '-' antes: {guiones_antes}")
    print(f"  Valores '-' después: {guiones_despues}")
    
    if guiones_despues == 0:
        print("  ✓ VALIDACIÓN EXITOSA: Todos los '-' eliminados")
    else:
        print("  ✗ VALIDACIÓN FALLIDA: Aún quedan valores '-'")
    
    return guiones_despues == 0


# Ejecutar limpieza Problema 1
detectar_problema_1(df_raw)
df_step1 = limpiar_problema_1(df_raw)
validar_problema_1(df_raw, df_step1)


PROBLEMA 1: Detección de valores '-' como nulos
  - dep_date: 861 valores '-'
  - dep_time: 861 valores '-'
  - dep_terminal: 5846 valores '-'
  - dep_status: 868 valores '-'
  - dep_weather_min: 861 valores '-'
  - dep_weather_max: 861 valores '-'
  - dep_weather_desc: 861 valores '-'
  - dep_counter: 861 valores '-'
  - dep_door: 861 valores '-'
  - arr_date: 12920 valores '-'
  - arr_time: 12920 valores '-'
  - arr_terminal: 19076 valores '-'
  - arr_status: 12927 valores '-'
  - arr_weather_min: 12920 valores '-'
  - arr_weather_max: 12920 valores '-'
  - arr_weather_desc: 12920 valores '-'
  - arr_room: 12920 valores '-'
  - arr_belt: 12920 valores '-'

✓ Valores '-' reemplazados por NaN

--------------------------------------------------------------------------------
VALIDACIÓN PROBLEMA 1
--------------------------------------------------------------------------------
  Valores '-' antes: 135184
  Valores '-' después: 0
  ✓ VALIDACIÓN EXITOSA: Todos los '-' eliminados


True

In [5]:
# ============================================================================
# ETAPA 3: LIMPIEZA - PROBLEMA 2 (Duplicados)
# ============================================================================

def detectar_problema_2(df):
    """
    Detecta filas duplicadas
    """
    print("\n" + "="*80)
    print("PROBLEMA 2: Detección de duplicados")
    print("="*80)
    
    duplicados_totales = df.duplicated().sum()
    print(f"  Total de filas duplicadas: {duplicados_totales}")
    
    if duplicados_totales > 0:
        print("\n  Ejemplos de duplicados:")
        display(df[df.duplicated(keep=False)].sort_values('flightNumber').head(6))
    
    return duplicados_totales


def limpiar_problema_2(df):
    """
    Elimina filas duplicadas manteniendo la primera ocurrencia
    """
    df_clean = df.copy()
    filas_antes = len(df_clean)
    df_clean.drop_duplicates(inplace=True, keep='first')
    filas_despues = len(df_clean)
    
    print(f"\n✓ Duplicados eliminados: {filas_antes - filas_despues} filas")
    return df_clean


def validar_problema_2(df_limpio):
    """
    Valida que no queden duplicados
    """
    print("\n" + "-"*80)
    print("VALIDACIÓN PROBLEMA 2")
    print("-"*80)
    
    duplicados = df_limpio.duplicated().sum()
    print(f"  Duplicados restantes: {duplicados}")
    
    if duplicados == 0:
        print("  ✓ VALIDACIÓN EXITOSA: No hay duplicados")
    else:
        print("  ✗ VALIDACIÓN FALLIDA: Aún hay duplicados")
    
    return duplicados == 0


# Ejecutar limpieza Problema 2
detectar_problema_2(df_step1)
df_step2 = limpiar_problema_2(df_step1)
validar_problema_2(df_step2)


PROBLEMA 2: Detección de duplicados
  Total de filas duplicadas: 24

  Ejemplos de duplicados:


,flightNumber,plane,dep_date,dep_time,dep_airport_name,dep_airport_code,dep_terminal,dep_status,dep_weather_min,dep_weather_max,...,arr_airport_name,arr_airport_code,arr_terminal,arr_status,arr_weather_min,arr_weather_max,arr_weather_desc,arr_room,arr_belt,timestamp
15589,AEA5001,ATR-72,14/04/18,09:20,MALAGA-COSTA DEL SOL,AGP,3,El vuelo ha despegado a las 09:14,11,21,...,MELILLA,MLN,T,Llegada prevista a las 09:50,12,19,Nubosidad variable,SLL,1,2018-04-14 09:29:25
15722,AEA5001,ATR-72,14/04/18,09:20,MALAGA-COSTA DEL SOL,AGP,3,El vuelo ha despegado a las 09:14,11,21,...,MELILLA,MLN,T,Llegada prevista a las 09:50,12,19,Nubosidad variable,SLL,1,2018-04-14 09:29:25
14349,AEA6006,BOEING 737-800 WINGLETS,14/04/18,07:00,PALMA DE MALLORCA,PMI,N,El vuelo ha despegado a las 07:00,11,15,...,BARCELONA-EL PRAT,BCN,T1,Llegada prevista a las 07:41,13,17,Lluvia,T1_G,03,2018-04-14 07:26:52
14271,AEA6006,BOEING 737-800 WINGLETS,14/04/18,07:00,PALMA DE MALLORCA,PMI,N,El vuelo ha despegado a las 07:00,11,15,...,BARCELONA-EL PRAT,BCN,T1,Llegada prevista a las 07:41,13,17,Lluvia,T1_G,03,2018-04-14 07:26:52
18253,EVE1123,32A,14/04/18,12:50,ALICANTE-ELCHE,ALC,N,El vuelo ha despegado a las 12:57,10,22,...,VIGO,VGO,1,Llegada prevista a las 14:30,6,16,Nubosidad variable,S1,3,2018-04-14 13:34:00
18178,EVE1123,32A,14/04/18,12:50,ALICANTE-ELCHE,ALC,N,El vuelo ha despegado a las 12:57,10,22,...,VIGO,VGO,1,Llegada prevista a las 14:30,6,16,Nubosidad variable,S1,3,2018-04-14 13:34:00



✓ Duplicados eliminados: 24 filas

--------------------------------------------------------------------------------
VALIDACIÓN PROBLEMA 2
--------------------------------------------------------------------------------
  Duplicados restantes: 0
  ✓ VALIDACIÓN EXITOSA: No hay duplicados


True

In [6]:
# ============================================================================
# ETAPA 4: LIMPIEZA - PROBLEMA 3 (Inconsistencias en códigos de aeropuerto)
# ============================================================================

def detectar_problema_3(df):
    """
    Detecta códigos de aeropuerto con longitud incorrecta
    """
    print("\n" + "="*80)
    print("PROBLEMA 3: Detección de códigos de aeropuerto inconsistentes")
    print("="*80)
    
    problemas = {}
    
    for col in ['dep_airport_code', 'arr_airport_code']:
        if col in df.columns:
            longitudes = df[col].dropna().str.len()
            incorrectos = (longitudes != 3).sum()
            
            if incorrectos > 0:
                problemas[col] = incorrectos
                print(f"\n  Columna: {col}")
                print(f"    - Códigos con longitud != 3: {incorrectos}")
                print(f"    - Ejemplos:")
                ejemplos = df[df[col].str.len() != 3][col].unique()[:5]
                for ej in ejemplos:
                    print(f"      '{ej}' (longitud: {len(ej)})")
    
    return problemas


def limpiar_problema_3(df):
    """
    Normaliza códigos de aeropuerto a 3 caracteres en mayúsculas
    """
    df_clean = df.copy()
    
    for col in ['dep_airport_code', 'arr_airport_code']:
        if col in df_clean.columns:
            # Convertir a mayúsculas y eliminar espacios
            df_clean[col] = df_clean[col].str.strip().str.upper()
            
            # Truncar o rellenar a 3 caracteres (ajustar según necesidad)
            # Aquí asumimos que códigos con longitud != 3 son errores y se marcan como nulos
            mask = df_clean[col].str.len() != 3
            df_clean.loc[mask, col] = np.nan
    
    print("\n✓ Códigos de aeropuerto normalizados a 3 caracteres en mayúsculas")
    return df_clean


def validar_problema_3(df_limpio):
    """
    Valida que todos los códigos tengan 3 caracteres
    """
    print("\n" + "-"*80)
    print("VALIDACIÓN PROBLEMA 3")
    print("-"*80)
    
    todo_ok = True
    
    for col in ['dep_airport_code', 'arr_airport_code']:
        if col in df_limpio.columns:
            longitudes = df_limpio[col].dropna().str.len()
            incorrectos = (longitudes != 3).sum()
            print(f"  {col}: {incorrectos} códigos con longitud != 3")
            
            if incorrectos > 0:
                todo_ok = False
    
    if todo_ok:
        print("  ✓ VALIDACIÓN EXITOSA: Todos los códigos tienen 3 caracteres")
    else:
        print("  ✗ VALIDACIÓN FALLIDA: Aún hay códigos incorrectos")
    
    return todo_ok


# Ejecutar limpieza Problema 3
detectar_problema_3(df_step2)
df_step3 = limpiar_problema_3(df_step2)
validar_problema_3(df_step3)


PROBLEMA 3: Detección de códigos de aeropuerto inconsistentes

✓ Códigos de aeropuerto normalizados a 3 caracteres en mayúsculas

--------------------------------------------------------------------------------
VALIDACIÓN PROBLEMA 3
--------------------------------------------------------------------------------
  dep_airport_code: 0 códigos con longitud != 3
  arr_airport_code: 0 códigos con longitud != 3
  ✓ VALIDACIÓN EXITOSA: Todos los códigos tienen 3 caracteres


True

In [7]:
# ============================================================================
# ETAPA 5: LIMPIEZA - PROBLEMA 4 (Inconsistencias en nombres de aeropuerto)
# ============================================================================

def detectar_problema_4(df):
    """
    Detecta inconsistencias en nombres de aeropuertos para el mismo código
    """
    print("\n" + "="*80)
    print("PROBLEMA 4: Detección de inconsistencias en nombres de aeropuerto")
    print("="*80)
    
    # Analizar salidas
    dep_mapping = df.groupby('dep_airport_code')['dep_airport_name'].unique()
    dep_inconsistentes = {k: v for k, v in dep_mapping.items() if len(v) > 1}
    
    # Analizar llegadas
    arr_mapping = df.groupby('arr_airport_code')['arr_airport_name'].unique()
    arr_inconsistentes = {k: v for k, v in arr_mapping.items() if len(v) > 1}
    
    print(f"\n  Códigos de salida con múltiples nombres: {len(dep_inconsistentes)}")
    for code, names in list(dep_inconsistentes.items())[:3]:
        print(f"    {code}: {names}")
    
    print(f"\n  Códigos de llegada con múltiples nombres: {len(arr_inconsistentes)}")
    for code, names in list(arr_inconsistentes.items())[:3]:
        print(f"    {code}: {names}")
    
    return dep_inconsistentes, arr_inconsistentes


def limpiar_problema_4(df):
    """
    Normaliza nombres de aeropuertos usando el nombre más frecuente por código
    """
    df_clean = df.copy()
    
    # Crear mapeo código -> nombre más frecuente (salidas)
    dep_map = df_clean.groupby('dep_airport_code')['dep_airport_name'].agg(
        lambda x: x.value_counts().index[0] if len(x) > 0 else np.nan
    ).to_dict()
    
    # Crear mapeo código -> nombre más frecuente (llegadas)
    arr_map = df_clean.groupby('arr_airport_code')['arr_airport_name'].agg(
        lambda x: x.value_counts().index[0] if len(x) > 0 else np.nan
    ).to_dict()
    
    # Aplicar mapeos
    df_clean['dep_airport_name'] = df_clean['dep_airport_code'].map(dep_map)
    df_clean['arr_airport_name'] = df_clean['arr_airport_code'].map(arr_map)
    
    print("\n✓ Nombres de aeropuertos normalizados según código")
    return df_clean


def validar_problema_4(df_limpio):
    """
    Valida que cada código tenga un único nombre
    """
    print("\n" + "-"*80)
    print("VALIDACIÓN PROBLEMA 4")
    print("-"*80)
    
    dep_mapping = df_limpio.groupby('dep_airport_code')['dep_airport_name'].nunique()
    arr_mapping = df_limpio.groupby('arr_airport_code')['arr_airport_name'].nunique()
    
    dep_inconsistentes = (dep_mapping > 1).sum()
    arr_inconsistentes = (arr_mapping > 1).sum()
    
    print(f"  Códigos de salida con múltiples nombres: {dep_inconsistentes}")
    print(f"  Códigos de llegada con múltiples nombres: {arr_inconsistentes}")
    
    if dep_inconsistentes == 0 and arr_inconsistentes == 0:
        print("  ✓ VALIDACIÓN EXITOSA: Cada código tiene un único nombre")
        return True
    else:
        print("  ✗ VALIDACIÓN FALLIDA: Aún hay inconsistencias")
        return False


# Ejecutar limpieza Problema 4
detectar_problema_4(df_step3)
df_step4 = limpiar_problema_4(df_step3)
validar_problema_4(df_step4)


PROBLEMA 4: Detección de inconsistencias en nombres de aeropuerto

  Códigos de salida con múltiples nombres: 0

  Códigos de llegada con múltiples nombres: 0

✓ Nombres de aeropuertos normalizados según código

--------------------------------------------------------------------------------
VALIDACIÓN PROBLEMA 4
--------------------------------------------------------------------------------
  Códigos de salida con múltiples nombres: 0
  Códigos de llegada con múltiples nombres: 0
  ✓ VALIDACIÓN EXITOSA: Cada código tiene un único nombre


True

In [8]:
# ============================================================================
# ETAPA 6: LIMPIEZA - PROBLEMA 5 (Formatos de fecha y hora inconsistentes)
# ============================================================================

def detectar_problema_5(df):
    """
    Detecta problemas en formatos de fecha y hora
    """
    print("\n" + "="*80)
    print("PROBLEMA 5: Detección de formatos de fecha/hora inconsistentes")
    print("="*80)
    
    print("\n  Ejemplos de fechas y horas actuales:")
    print(f"    dep_date: {df['dep_date'].head(3).tolist()}")
    print(f"    dep_time: {df['dep_time'].head(3).tolist()}")
    print(f"    arr_date: {df['arr_date'].head(3).tolist()}")
    print(f"    arr_time: {df['arr_time'].head(3).tolist()}")
    
    print("\n  Problema: Fechas en formato DD/MM/YY y horas separadas")
    print("  Solución: Unificar en datetime ISO 8601 (YYYY-MM-DDTHH:MM:SS)")


def limpiar_problema_5(df):
    """
    Convierte fechas y horas a formato ISO 8601
    """
    df_clean = df.copy()
    
    # Función auxiliar para combinar fecha y hora
    def combinar_fecha_hora(fecha, hora):
        try:
            # Parsear fecha DD/MM/YY
            fecha_obj = pd.to_datetime(fecha, format='%d/%m/%y', errors='coerce')
            
            # Combinar con hora
            if pd.notna(fecha_obj) and pd.notna(hora):
                datetime_str = f"{fecha_obj.strftime('%Y-%m-%d')}T{hora}:00"
                return datetime_str
            else:
                return None
        except:
            return None
    
    # Crear campos datetime unificados
    df_clean['dep_datetime_scheduled'] = df_clean.apply(
        lambda row: combinar_fecha_hora(row['dep_date'], row['dep_time']), axis=1
    )
    
    df_clean['arr_datetime_scheduled'] = df_clean.apply(
        lambda row: combinar_fecha_hora(row['arr_date'], row['arr_time']), axis=1
    )
    
    # Convertir timestamp de ingesta
    df_clean['ingestion_timestamp'] = pd.to_datetime(
        df_clean['timestamp'], errors='coerce'
    ).dt.strftime('%Y-%m-%dT%H:%M:%S')
    
    print("\n✓ Fechas y horas convertidas a formato ISO 8601")
    return df_clean


def validar_problema_5(df_limpio):
    """
    Valida que las fechas estén en formato ISO 8601
    """
    print("\n" + "-"*80)
    print("VALIDACIÓN PROBLEMA 5")
    print("-"*80)
    
    print("\n  Ejemplos de datetime unificados:")
    print(f"    dep_datetime_scheduled: {df_limpio['dep_datetime_scheduled'].head(3).tolist()}")
    print(f"    arr_datetime_scheduled: {df_limpio['arr_datetime_scheduled'].head(3).tolist()}")
    
    # Validar formato ISO 8601
    patron_iso = r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}$'
    
    dep_validos = df_limpio['dep_datetime_scheduled'].dropna().str.match(patron_iso).all()
    arr_validos = df_limpio['arr_datetime_scheduled'].dropna().str.match(patron_iso).all()
    
    if dep_validos and arr_validos:
        print("  ✓ VALIDACIÓN EXITOSA: Todas las fechas en formato ISO 8601")
        return True
    else:
        print("  ✗ VALIDACIÓN FALLIDA: Hay fechas con formato incorrecto")
        return False


# Ejecutar limpieza Problema 5
detectar_problema_5(df_step4)
df_step5 = limpiar_problema_5(df_step4)
validar_problema_5(df_step5)


PROBLEMA 5: Detección de formatos de fecha/hora inconsistentes

  Ejemplos de fechas y horas actuales:
    dep_date: ['13/04/18', '13/04/18', '13/04/18']
    dep_time: ['10:00', '14:15', '15:05']
    arr_date: ['13/04/18', '13/04/18', '13/04/18']
    arr_time: ['10:10', '14:30', '15:15']

  Problema: Fechas en formato DD/MM/YY y horas separadas
  Solución: Unificar en datetime ISO 8601 (YYYY-MM-DDTHH:MM:SS)

✓ Fechas y horas convertidas a formato ISO 8601

--------------------------------------------------------------------------------
VALIDACIÓN PROBLEMA 5
--------------------------------------------------------------------------------

  Ejemplos de datetime unificados:
    dep_datetime_scheduled: ['2018-04-13T10:00:00', '2018-04-13T14:15:00', '2018-04-13T15:05:00']
    arr_datetime_scheduled: ['2018-04-13T10:10:00', '2018-04-13T14:30:00', '2018-04-13T15:15:00']
  ✓ VALIDACIÓN EXITOSA: Todas las fechas en formato ISO 8601


True

In [9]:
# ============================================================================
# ETAPA 7: LIMPIEZA - PROBLEMA 6 (Estados de vuelo no estructurados)
# ============================================================================

def detectar_problema_6(df):
    """
    Detecta estados de vuelo en texto libre
    """
    print("\n" + "="*80)
    print("PROBLEMA 6: Detección de estados de vuelo no estructurados")
    print("="*80)
    
    print("\n  Ejemplos de dep_status:")
    print(df['dep_status'].value_counts().head(5))
    
    print("\n  Ejemplos de arr_status:")
    print(df['arr_status'].value_counts().head(5))
    
    print("\n  Problema: Estados en texto libre sin categorización")
    print("  Solución: Extraer código de estado normalizado")


def limpiar_problema_6(df):
    """
    Extrae códigos de estado normalizados y datetime_actual de los textos de estado
    """
    df_clean = df.copy()
    
    # Función para extraer código de estado
    def extraer_status_code(texto):
        if pd.isna(texto):
            return 'DESCONOCIDO'
        
        texto_lower = texto.lower()
        
        if 'despegado' in texto_lower or 'ha despegado' in texto_lower:
            return 'DESPEGADO'
        elif 'aterrizado' in texto_lower or 'ha aterrizado' in texto_lower:
            return 'ATERRIZADO'
        elif 'cancelado' in texto_lower or 'cancelada' in texto_lower:
            return 'CANCELADO'
        elif 'retrasado' in texto_lower or 'retraso' in texto_lower:
            return 'RETRASADO'
        elif 'prevista' in texto_lower or 'previsto' in texto_lower:
            return 'PROGRAMADO'
        else:
            return 'OTRO'
    
    # Función para extraer hora actual del texto de estado
    def extraer_hora_actual(texto, fecha_base):
        if pd.isna(texto):
            return None
        
        # Buscar patrón "a las HH:MM"
        match = re.search(r'a las (\d{1,2}):(\d{2})', texto)
        if match and pd.notna(fecha_base):
            hora = match.group(1).zfill(2)
            minuto = match.group(2)
            # Extraer solo la fecha de fecha_base
            fecha_parte = fecha_base.split('T')[0]
            return f"{fecha_parte}T{hora}:{minuto}:00"
        return None
    
    # Aplicar extracciones
    df_clean['dep_status_code'] = df_clean['dep_status'].apply(extraer_status_code)
    df_clean['arr_status_code'] = df_clean['arr_status'].apply(extraer_status_code)
    
    df_clean['dep_datetime_actual'] = df_clean.apply(
        lambda row: extraer_hora_actual(row['dep_status'], row['dep_datetime_scheduled']),
        axis=1
    )
    
    df_clean['arr_datetime_actual'] = df_clean.apply(
        lambda row: extraer_hora_actual(row['arr_status'], row['arr_datetime_scheduled']),
        axis=1
    )
    
    print("\n✓ Códigos de estado extraídos y datetime_actual generados")
    return df_clean


def validar_problema_6(df_limpio):
    """
    Valida que los códigos de estado sean válidos
    """
    print("\n" + "-"*80)
    print("VALIDACIÓN PROBLEMA 6")
    print("-"*80)
    
    codigos_validos = {'PROGRAMADO', 'DESPEGADO', 'ATERRIZADO', 'CANCELADO', 'RETRASADO', 'OTRO', 'DESCONOCIDO'}
    
    print("\n  Distribución de dep_status_code:")
    print(df_limpio['dep_status_code'].value_counts())
    
    print("\n  Distribución de arr_status_code:")
    print(df_limpio['arr_status_code'].value_counts())
    
    dep_invalidos = ~df_limpio['dep_status_code'].isin(codigos_validos)
    arr_invalidos = ~df_limpio['arr_status_code'].isin(codigos_validos)
    
    if not dep_invalidos.any() and not arr_invalidos.any():
        print("\n  ✓ VALIDACIÓN EXITOSA: Todos los códigos son válidos")
        return True
    else:
        print("\n  ✗ VALIDACIÓN FALLIDA: Hay códigos inválidos")
        return False


# Ejecutar limpieza Problema 6
detectar_problema_6(df_step5)
df_step6 = limpiar_problema_6(df_step5)
validar_problema_6(df_step6)


PROBLEMA 6: Detección de estados de vuelo no estructurados

  Ejemplos de dep_status:
dep_status
Salida prevista a las 07:00    546
Salida prevista a las 16:00    397
Salida prevista a las 07:30    388
Salida prevista a las 08:00    358
Salida prevista a las 10:30    318
Name: count, dtype: int64

  Ejemplos de arr_status:
arr_status
Llegada prevista a las 08:30    328
Llegada prevista a las 17:30    266
Llegada prevista a las 15:30    245
Llegada prevista a las 10:10    240
Llegada prevista a las 09:00    231
Name: count, dtype: int64

  Problema: Estados en texto libre sin categorización
  Solución: Extraer código de estado normalizado

✓ Códigos de estado extraídos y datetime_actual generados

--------------------------------------------------------------------------------
VALIDACIÓN PROBLEMA 6
--------------------------------------------------------------------------------

  Distribución de dep_status_code:
dep_status_code
PROGRAMADO     25317
DESPEGADO      12582
DESCONOCIDO    

True

In [10]:
# ============================================================================
# ETAPA 8: LIMPIEZAS ADICIONALES (Completitud y consistencia)
# ============================================================================

def limpiezas_adicionales(df):
    """
    Aplica limpiezas adicionales para mejorar calidad
    """
    print("\n" + "="*80)
    print("LIMPIEZAS ADICIONALES")
    print("="*80)
    
    df_clean = df.copy()
    
    # 1. Normalizar nombres de columnas
    print("\n1. Normalizando nombres de columnas...")
    # (Ya están bien, pero se podría hacer snake_case completo)
    
    # 2. Convertir temperaturas a numérico
    print("2. Convirtiendo temperaturas a tipo numérico...")
    for col in ['dep_weather_min', 'dep_weather_max', 'arr_weather_min', 'arr_weather_max']:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # 3. Eliminar espacios en blanco de strings
    print("3. Eliminando espacios en blanco innecesarios...")
    string_cols = df_clean.select_dtypes(include=['object']).columns
    for col in string_cols:
        df_clean[col] = df_clean[col].str.strip() if df_clean[col].dtype == 'object' else df_clean[col]
    
    # 4. Validar que flightNumber no sea nulo
    print("4. Validando integridad de flightNumber...")
    nulos_flight = df_clean['flightNumber'].isna().sum()
    if nulos_flight > 0:
        print(f"   ⚠ Advertencia: {nulos_flight} filas sin flightNumber (se eliminarán)")
        df_clean = df_clean[df_clean['flightNumber'].notna()]
    
    print("\n✓ Limpiezas adicionales completadas")
    return df_clean

def normalizar_nulos(df):
    """
    Reemplaza NaN y valores '-' que significan 'sin dato' por None,
    para que el resultante use 'null' correctamente.
    """
    df_clean = df.copy()
    
    # Columnas donde '-' significa 'sin dato'
    columnas_guion_a_null = [
        'dep_terminal', 'dep_counter', 'dep_door',
        'arr_terminal', 'arr_room', 'arr_belt'
    ]
    
    for col in columnas_guion_a_null:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].replace('-', np.nan)
    
    # (Opcional) si quieres, puedes aplicar esto a otras columnas de texto
    # donde '-' NO es un valor real
    return df_clean

# Ejecutar limpiezas adicionales
df_final = limpiezas_adicionales(df_step6)
df_final = normalizar_nulos(df_final)


LIMPIEZAS ADICIONALES

1. Normalizando nombres de columnas...
2. Convirtiendo temperaturas a tipo numérico...
3. Eliminando espacios en blanco innecesarios...
4. Validando integridad de flightNumber...

✓ Limpiezas adicionales completadas


In [11]:
# ============================================================================
# ETAPA 9: RESUMEN FINAL DE CALIDAD
# ============================================================================

def generar_resumen_calidad(df_original, df_final):
    """
    Genera un resumen comparativo de la calidad de datos
    """
    print("\n" + "="*80)
    print("RESUMEN FINAL DE CALIDAD DE DATOS")
    print("="*80)
    
    print(f"\n1. Dimensiones:")
    print(f"   Original: {df_original.shape[0]} filas × {df_original.shape[1]} columnas")
    print(f"   Final:    {df_final.shape[0]} filas × {df_final.shape[1]} columnas")
    print(f"   Filas eliminadas: {df_original.shape[0] - df_final.shape[0]}")
    
    print(f"\n2. Valores nulos:")
    print(f"   Original: {df_original.isnull().sum().sum()} nulos")
    print(f"   Final:    {df_final.isnull().sum().sum()} nulos")
    
    print(f"\n3. Duplicados:")
    print(f"   Original: {df_original.duplicated().sum()}")
    print(f"   Final:    {df_final.duplicated().sum()}")
    
    print(f"\n4. Nuevas columnas creadas:")
    nuevas_cols = set(df_final.columns) - set(df_original.columns)
    for col in nuevas_cols:
        print(f"   - {col}")
    
    print("\n✓ Dataset limpio y listo para exportación")


# Generar resumen
generar_resumen_calidad(df_raw, df_final)


RESUMEN FINAL DE CALIDAD DE DATOS

1. Dimensiones:
   Original: 39102 filas × 25 columnas
   Final:    39078 filas × 32 columnas
   Filas eliminadas: 24

2. Valores nulos:
   Original: 0 nulos
   Final:    180178 nulos

3. Duplicados:
   Original: 24
   Final:    0

4. Nuevas columnas creadas:
   - arr_status_code
   - arr_datetime_scheduled
   - dep_status_code
   - arr_datetime_actual
   - ingestion_timestamp
   - dep_datetime_actual
   - dep_datetime_scheduled

✓ Dataset limpio y listo para exportación


In [12]:
# ============================================================================
# ETAPA 10: EXPORTACIÓN A JSON
# ============================================================================

def safe_value(v):
    """
    Convierte valores NaN de pandas a None (que será null en JSON).
    Deja el resto tal cual.
    """
    if pd.isna(v):
        return None
    return v


def transformar_a_json_estructura(df):
    """
    Transforma el DataFrame limpio a la estructura JSON propuesta
    
    Returns:
        Lista de diccionarios con la estructura JSON de vuelos
    """
    vuelos_json = []
    
    for _, row in df.iterrows():
        vuelo = {
            "flight_number": safe_value(row.get('flightNumber')),
            "plane": safe_value(row.get('plane')),
            "departure": {
                "airport_name": safe_value(row.get('dep_airport_name')),
                "airport_code": safe_value(row.get('dep_airport_code')),
                "terminal": safe_value(row.get('dep_terminal')),
                "counter": safe_value(row.get('dep_counter')),
                "gate": safe_value(row.get('dep_door')),
                "datetime_scheduled": safe_value(row.get('dep_datetime_scheduled')),
                "datetime_actual": safe_value(row.get('dep_datetime_actual')),
                "status_code": safe_value(row.get('dep_status_code')),
                "status_text": safe_value(row.get('dep_status'))
            },
            "arrival": {
                "airport_name": safe_value(row.get('arr_airport_name')),
                "airport_code": safe_value(row.get('arr_airport_code')),
                "terminal": safe_value(row.get('arr_terminal')),
                "room": safe_value(row.get('arr_room')),
                "belt": safe_value(row.get('arr_belt')),
                "datetime_scheduled": safe_value(row.get('arr_datetime_scheduled')),
                "datetime_actual": safe_value(row.get('arr_datetime_actual')),
                "status_code": safe_value(row.get('arr_status_code')),
                "status_text": safe_value(row.get('arr_status'))
            },
            "weather": {
                "departure": {
                    "temp_min": safe_value(row.get('dep_weather_min')),
                    "temp_max": safe_value(row.get('dep_weather_max')),
                    "description": safe_value(row.get('dep_weather_desc'))
                },
                "arrival": {
                    "temp_min": safe_value(row.get('arr_weather_min')),
                    "temp_max": safe_value(row.get('arr_weather_max')),
                    "description": safe_value(row.get('arr_weather_desc'))
                }
            },
            "ingestion_timestamp": safe_value(row.get('ingestion_timestamp'))
        }
        
        vuelos_json.append(vuelo)
    
    return vuelos_json


def exportar_json(vuelos_json, ruta_salida='vuelos_limpio.json', num_documentos=None, indent=2):
    """
    Exporta la lista de vuelos a un archivo JSON
    
    Args:
        vuelos_json: lista de diccionarios con estructura de vuelos
        ruta_salida: ruta del archivo JSON de salida
        num_documentos: número de documentos a exportar. Si es None, exporta todos.
        indent: nivel de indentación para legibilidad
    """
    # Determinar cuántos documentos exportar
    if num_documentos is None:
        documentos_a_exportar = vuelos_json
        total_exportado = len(vuelos_json)
    else:
        documentos_a_exportar = vuelos_json[:num_documentos]
        total_exportado = min(num_documentos, len(vuelos_json))
    
    # Exportar a JSON
    with open(ruta_salida, 'w', encoding='utf-8') as f:
        json.dump(documentos_a_exportar, f, ensure_ascii=False, indent=indent)
    
    print(f"\n✓ Archivo JSON exportado: {ruta_salida}")
    print(f"  Total de vuelos exportados: {total_exportado}")
    print(f"  Total de vuelos disponibles: {len(vuelos_json)}")
    
    if num_documentos and num_documentos < len(vuelos_json):
        print(f"  ⚠ Se exportaron solo los primeros {num_documentos} documentos")
    
    tamanio_kb = round(len(json.dumps(documentos_a_exportar)) / 1024, 2)
    print(f"  Tamaño del archivo: {tamanio_kb} KB")


# Ejecutar transformación y exportación
vuelos_json = transformar_a_json_estructura(df_final)

In [13]:
# 1. Exportar TODOS los documentos (comportamiento por defecto)
#exportar_json(vuelos_json, ruta_salida='dataset3_vuelos_completo.json')

# 2. Exportar solo los primeros 10 documentos (para pruebas)
#exportar_json(vuelos_json, ruta_salida='dataset3_vuelos_muestra.json', num_documentos=10)

# 3. Exportar los primeros 100 documentos
exportar_json(vuelos_json, ruta_salida='dataset3_vuelos_100.json', num_documentos=100)

# 4. Exportar 1 solo documento (para validación rápida)
#exportar_json(vuelos_json, ruta_salida='dataset3_vuelos_ejemplo.json', num_documentos=1)


✓ Archivo JSON exportado: dataset3_vuelos_100.json
  Total de vuelos exportados: 100
  Total de vuelos disponibles: 39078
  ⚠ Se exportaron solo los primeros 100 documentos
  Tamaño del archivo: 80.44 KB


In [14]:
# ============================================================================
# FIN DEL NOTEBOOK
# ============================================================================

print("\n" + "="*80)
print("PROCESO COMPLETADO EXITOSAMENTE")
print("="*80)
print("\nArchivo generado: dataset3_vuelos_limpio.json")
print("El dataset está listo para la entrega.")


PROCESO COMPLETADO EXITOSAMENTE

Archivo generado: dataset3_vuelos_limpio.json
El dataset está listo para la entrega.


In [15]:
df_final.sample()

,flightNumber,plane,dep_date,dep_time,dep_airport_name,dep_airport_code,dep_terminal,dep_status,dep_weather_min,dep_weather_max,...,arr_room,arr_belt,timestamp,dep_datetime_scheduled,arr_datetime_scheduled,ingestion_timestamp,dep_status_code,arr_status_code,dep_datetime_actual,arr_datetime_actual
33811,IBS3927,32A,15/04/18,06:45,TENERIFE SUR,TFS,T,El vuelo ha despegado a las 06:49,17.0,23.0,...,10,18,2018-04-15 12:35:49,2018-04-15T06:45:00,2018-04-15T10:30:00,2018-04-15T12:35:49,DESPEGADO,ATERRIZADO,2018-04-15T06:49:00,2018-04-15T10:34:00
